In [8]:
# Import necessary libraries
import cv2
import numpy as np
import os


# Define augmentation functions
# Define the function to apply rotation
def rotate_image(image, angle):
    image_center = (image.shape[1] // 2, image.shape[0] // 2)
    rotation_matrix = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
    return rotated_image


def apply_super_resolution(image):
    upscaled_image = cv2.resize(image, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_LINEAR)
    return upscaled_image

def apply_optical_distortion(image):
    k1, k2, p1, p2, k3 = -0.2, 0.1, 0.0, 0.0, 0.0
    fx, fy, cx, cy = 500, 500, 320, 240
    dist_coeffs = np.array([k1, k2, p1, p2, k3])
    camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    new_camera_matrix, _ = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, image.shape[:2], 1)
    distorted_image = cv2.undistort(image, camera_matrix, dist_coeffs, None, new_camera_matrix)
    return distorted_image

def apply_dithering_and_quantization(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, quantized_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY)
    return quantized_image

def apply_spatial_augmentation(image):
    dx, dy = np.random.randint(-20, 20), np.random.randint(-20, 20)
    angle = np.random.randint(-30, 30)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    translated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    rotated_image = cv2.warpAffine(translated_image, M, (image.shape[1], image.shape[0]))
    return rotated_image


def add_fog(image, intensity):
    overlay = np.ones_like(image) * 255
    overlay = cv2.addWeighted(overlay, intensity, np.zeros_like(image), 1 - intensity, 0)
    foggy_image = cv2.addWeighted(image, 1 - intensity, overlay, intensity, 0)
    return foggy_image




In [9]:
# Define paths and parameters
dataset_dir = r"C:\Users\User\Desktop\project\Leaf Images Database"
output_dir = r"C:\Users\User\Desktop\project\augmented_images"
input_size = (224, 224)
angles = [180]  


In [10]:

# Process images and apply augmentations
for subfolder in os.listdir(dataset_dir):
    subfolder_output_dir = os.path.join(output_dir, subfolder)
    os.makedirs(subfolder_output_dir, exist_ok=True)

    for root, _, files in os.walk(os.path.join(dataset_dir, subfolder)):
        for filename in files:
            if filename.endswith(('.png', '.jpg','.JPG', '.jpeg', '.bmp')):
                image_path = os.path.join(root, filename)
                image = cv2.imread(image_path)

                # Normalize the image
                normalized_image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
                resized_image = cv2.resize(normalized_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'normalized_image{filename}'), resized_image)
                
                 # Apply augmentations to the normalized image 
                for angle in angles:
                    rotated_image = rotate_image(normalized_image, angle)
                    resized_image = cv2.resize(rotated_image, input_size)
                    output_filename = os.path.join(subfolder_output_dir, f'rotated_image_{angle}_{filename}')
                    cv2.imwrite(output_filename, resized_image)


               
                translated_image = cv2.warpAffine(normalized_image, np.float32([[1, 0, 100], [0, 1, 50]]), (normalized_image.shape[1], normalized_image.shape[0]))
                resized_image = cv2.resize(translated_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'translated_image_{filename}'), resized_image)

                rescaled_image = cv2.resize(normalized_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'rescaled_image_{filename}'), rescaled_image)

                flipped_image = cv2.flip(normalized_image, 1)
                resized_image = cv2.resize(flipped_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'flipped_image_{filename}'), resized_image)

                zoomed_image = cv2.resize(normalized_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'zoomed_image_{filename}'), zoomed_image)

                brightness_factor = 1.5
                brighter_image = np.clip(normalized_image * brightness_factor, 0, 255).astype(np.uint8)
                resized_image = cv2.resize(brighter_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'brighter_image_{filename}'), resized_image)

                contrast_factor = 1.5
                adjusted_image = np.clip((normalized_image - 127.5) * contrast_factor + 127.5, 0, 255).astype(np.uint8)
                resized_image = cv2.resize(adjusted_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'adjusted_image_{filename}'), resized_image)

                jitter_range = 20
                jitter = np.random.randint(-jitter_range, jitter_range + 1, size=3)
                jittered_image = np.clip(normalized_image + jitter, 0, 255).astype(np.uint8)
                resized_image = cv2.resize(jittered_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'jittered_image_{filename}'), resized_image)

                noise = np.random.normal(0, 20, normalized_image.shape).astype(np.uint8)
                noisy_image = np.clip(normalized_image + noise, 0, 255).astype(np.uint8)
                resized_image = cv2.resize(noisy_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'noisy_image_{filename}'), resized_image)

                blurred_image = cv2.GaussianBlur(normalized_image, (5, 5), 0)
                sharpened_image = cv2.addWeighted(normalized_image, 2, blurred_image, -1, 0)
                resized_image = cv2.resize(sharpened_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'sharpened_image_{filename}'), resized_image)

                super_res_image = apply_super_resolution(normalized_image)
                resized_image = cv2.resize(super_res_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'super_res_image_{filename}'), resized_image)

                spatial_augmentation_image = apply_spatial_augmentation(normalized_image)
                resized_image = cv2.resize(spatial_augmentation_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'spatially_aware_image_{filename}'), resized_image)

                foggy_image = add_fog(normalized_image, 0.3)
                resized_image = cv2.resize(foggy_image, input_size)
                cv2.imwrite(os.path.join(subfolder_output_dir, f'foggy_image_{filename}'), resized_image)

    print(f"Augmented images saved for {subfolder}")

print("Augmentation completed for all images.")


Augmented images saved for Alstonia scholaris
Augmented images saved for Citrus aurantiifolia
Augmented images saved for Jatropha
Augmented images saved for Mangifera indica
Augmented images saved for Ocimum basilicum
Augmented images saved for Platanus orientalis
Augmented images saved for Pomegranate
Augmented images saved for Pongamia Pinnata
Augmented images saved for Psidium guajava
Augmented images saved for Syzygium cumini
Augmented images saved for Terminalia arjuna
Augmentation completed for all images.
